# Part 2: Intro to Private Training with Remote Execution

**Note that this tutorial was originally designed part of [OpenMined PySyft-TensorFlow tutorials](https://github.com/OpenMined/PySyft-TensorFlow/tree/master/examples)**

In the last section, we learned about PointerTensors, which create the underlying infrastructure we need for privacy preserving Deep Learning. In this section, we're going to see how to use these basic tools to train our first deep learning model using remote execution.

### Why use remote execution?

Let's say you are an AI startup who wants to build a deep learning model to detect [diabetic retinopathy (DR)](https://ai.googleblog.com/2016/11/deep-learning-for-detection-of-diabetic.html), which is the fastest growing cause of blindness. Before training your model, the first step would be to acquire a dataset of retinopathy images with signs of DR. One approach could be to work with a hospital and ask them to send you a copy of this dataset. However because of the sensitivity of the patients' data, the hospital might be exposed to liability risks.


That's where remote execution comes into the picture. Instead of bringing training data to the model (a central server), you bring the model to the training data (wherever it may live). In this case, it would be the hospital.

The idea is that this allows whoever is creating the data to own the only permanent copy, and thus maintain control over who ever has access to it. Pretty cool, eh?

# Section 2.1 - Private Training on Fashion MNIST

For this tutorial, we will train a model on the [Fashion MNIST dataset](https://www.tensorflow.org/datasets/catalog/fashion_mnist) to classify images of clothing.

We can assume that we have a remote worker named Bob who owns the data.

In [1]:
import tensorflow as tf
import syft as sy

hook = sy.TensorFlowHook(tf)
bob = sy.VirtualWorker(hook, id="bob")

Let's download the Fashion MNIST data from `tensorflow_datasets`. In the cell below, we are converting the data from [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) to `tf.Tensor` in order to have the PySyft functionalities. 

Note that adding tf.data.Dataset support is on the roadmap. 

In [2]:
import tensorflow_datasets as tfds

dataset = tfds.load("fashion_mnist:3.0.0")

train_dataset, test_dataset = dataset['train'], dataset['test']

# we are generating this large batch to convert
# the tf.data.Dataset into tf.Tensor
train_dataset = next(iter(train_dataset.batch(60000)))
test_dataset = next(iter(test_dataset.batch(10000)))

x_train, y_train = train_dataset['image'], train_dataset['label']
x_test, y_test = test_dataset['image'], test_dataset['label']

x_train = tf.cast(x_train, tf.float32) / 255.0
x_test = tf.cast(x_test, tf.float32) / 255.0

As decribed in Part 1, we can send this data to Bob with the `send` method on the `tf.Tensor`. 

In [3]:
x_train_ptr = x_train.send(bob)
y_train_ptr = y_train.send(bob)

Excellent! We have everything to start experimenting. To train our model on Bob's machine, we just have to perform the following steps:

- Define a model, including optimizer and loss
- Send the model to Bob
- Start the training process
- Get the trained model back

Let's do it!

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(28, 28 ,1)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(62, activation='softmax')
])

# Compile with optimizer, loss and metrics
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Once you have defined your model, you can simply send it to Bob calling the `send` method. It's the exact same process as sending a tensor.

In [5]:
model_ptr = model.send(bob)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: /var/folders/1c/vs6gvqqs1lngfqs6fkjrcclm0000gn/T/tmpvoiw62h1/assets


INFO:tensorflow:Assets written to: /var/folders/1c/vs6gvqqs1lngfqs6fkjrcclm0000gn/T/tmpvoiw62h1/assets


In [6]:
model_ptr

(Wrapper)>[ObjectPointer | me:50838506757 -> bob:19824984963]

Now, we have a pointer pointing to the model on Bob's machine. We can validate that's the case by inspecting the attribute `_objects` on the virtual worker. 

In [7]:
bob._objects[model_ptr.id_at_location]

Everything is ready to start training our model on this remote dataset. You can call `fit` and pass `x_train_ptr` `y_train_ptr` which are pointing to Bob's data. Note that's the exact same interface as normal `tf.keras`.

In [8]:
model_ptr.fit(x_train_ptr, y_train_ptr, epochs=2, validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 12s 245us/sample - loss: 0.5102 - accuracy: 0.8210 - val_loss: 0.3453 - val_accuracy: 0.8737
Epoch 2/2
48000/48000 [==============================] - 11s 234us/sample - loss: 0.3237 - accuracy: 0.8839 - val_loss: 0.2813 - val_accuracy: 0.8992


Fantastic! you have trained your model acheiving an accuracy greater than 88%.

You can get your trained model back by just calling `get` on it. 

In [9]:
model_gotten = model_ptr.get()

model_gotten

INFO:tensorflow:Assets written to: /var/folders/1c/vs6gvqqs1lngfqs6fkjrcclm0000gn/T/tmp63zp9g4f/assets


INFO:tensorflow:Assets written to: /var/folders/1c/vs6gvqqs1lngfqs6fkjrcclm0000gn/T/tmp63zp9g4f/assets


It's good practice to see if your model can generalize by assessing its accuracy on an holdout dataset. You can simply call `evaluate`.

In [10]:
model_gotten.evaluate(x_test, y_test, verbose=2)

10000/1 - 1s - loss: 0.6186 - accuracy: 0.8909


[0.3122893062591553, 0.8909]

Boom! The model remotely trained on Bob's data is more than 87% accurate on this holdout dataset.

If your model doesn't fit into the Sequential paradigm, you can use Keras's functional API, or even subclass [tf.keras.Model](https://www.tensorflow.org/guide/keras/custom_layers_and_models#building_models) to create custom models.

In [11]:
class CustomModel(tf.keras.Model):

    def __init__(self, num_classes=10):
        super(CustomModel, self).__init__(name='custom_model')
        self.num_classes = num_classes

        self.conv_1 = Conv2D(16, 3, padding='same', activation='relu', input_shape=(28, 28 ,1))
        self.maxpool_1 = MaxPooling2D()
        self.conv_2 = Conv2D(32, 3, padding='same', activation='relu')
        self.maxpool_2 = MaxPooling2D()
        self.flatten = Flatten()
        self.dense_1 = Dense(128, activation='relu')
        self.dense_2 = Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv_1(inputs)
        x = self.maxpool_1(x)
        x = self.conv_2(x)
        x = self.maxpool_2(x)
        x = self.flatten(x)
        x = self.dense_1(x)
        return self.dense_2(x)
              
model = CustomModel(10)

# need to call the model on dummy data before sending it
# in order to set the input shape (required when saving to SavedModel)
model.predict(tf.ones([1, 28, 28, 1]))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_ptr = model.send(bob)

model_ptr.fit(x_train_ptr, y_train_ptr, epochs=2, validation_split=0.2)

INFO:tensorflow:Assets written to: /var/folders/1c/vs6gvqqs1lngfqs6fkjrcclm0000gn/T/tmprtukf9dn/assets


INFO:tensorflow:Assets written to: /var/folders/1c/vs6gvqqs1lngfqs6fkjrcclm0000gn/T/tmprtukf9dn/assets


Train on 48000 samples, validate on 12000 samples
Epoch 1/2
48000/48000 [==============================] - 11s 229us/sample - loss: 0.4514 - accuracy: 0.8399 - val_loss: 0.3187 - val_accuracy: 0.8848
Epoch 2/2
48000/48000 [==============================] - 11s 234us/sample - loss: 0.3056 - accuracy: 0.8899 - val_loss: 0.2942 - val_accuracy: 0.8886


## Well Done!

And voilà! We have trained a Deep Learning model on Bob's data by sending the model to him. Never in this process do we ever see or request access to the underlying training data! We preserve the privacy of Bob!!!

### Join OpenMined Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org)